In [1]:
import pandas as pd
import numpy as np
trans_dataset = pd.read_csv('D:/Uni Mannheim/Semester 2/Data Mining 2/DMC/transactions.csv', sep='|')
books_data = pd.read_csv('D:/Uni Mannheim/Semester 2/Data Mining 2/DMC/final_annotated_item_dataset.csv', sep=',')
trans_data = trans_dataset.copy()

In [2]:
trans_dataset.head(1) #if we check the count: we have 365143

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0


In [3]:
trans_data.drop_duplicates
#we are only interested in books that have been clicked, basketed, or ordered by more than one user.
#at the same time we are only interested in users that have clicked, basketed, or ordered more than one book. 
trans_data= trans_data[trans_data.duplicated(subset='sessionID', keep=False)]
trans_data= trans_data[trans_data.duplicated(subset='itemID', keep=False)] #at this point if we check the count, it decreses to 125233

In [4]:
#it is not important that a user clicked an item for 500 times or 2 times! so we convert all data to binary setting.
trans_data['click_0_1'] = np.sign(trans_data['click'])
trans_data['basket_0_1'] = np.sign(trans_data['basket'])
trans_data['order_0_1'] = np.sign(trans_data['order'])
trans_data.head(2)

,sessionID,itemID,click,basket,order,click_0_1,basket_0_1,order_0_1
7,7,14576,1,1,0,1,1,0
8,7,17731,2,1,0,1,1,0


In [5]:
#here we create three different dataframes for further analysis. If we check the count, the order data frame includes 6513 rows.
#the basket data frame includes 27528 rows, and click data frame includes 113656 rows.
order_data = trans_data[trans_data['order_0_1'] > 0]
basket_data = trans_data[trans_data['basket_0_1'] > 0]
click_data = trans_data[trans_data['click_0_1'] > 0]

In [79]:
#we will start with order data that refers to a more limited set of all transactions.
order_df = order_data.merge(books_data, how='left', on='itemID')
order_df = order_df.drop(['click', 'basket', 'order', 'click_0_1', 'basket_0_1', 'Unnamed: 0','title', 'subtopics', 'description', 'interest_age', 'LangDetect', 'ISBN_13'], axis=1)

In [80]:
order_df.head(1)

,sessionID,itemID,order_0_1,author,publisher,main topic,language
0,104,74094,1,Agnes Spiecker,Hauschka Verlag,WDK,de


In [81]:
order_df= order_df[order_df['language'] == 'de']
order_df.dropna(axis=0, subset=['sessionID'], inplace=True)

In [82]:
order_df.count()
#interestingly more than 80% of books that have been ordered by more than one user are written in German!

sessionID     5279
itemID        5279
order_0_1     5279
author        4838
publisher     5279
main topic    5272
language      5279
dtype: int64

In [83]:
order_df.dropna(subset=['main topic'])
order_df['first topic'] = order_df['main topic'].astype(str).str[0]
order_df.head()

,sessionID,itemID,order_0_1,author,publisher,main topic,language,first topic
0,104,74094,1,Agnes Spiecker,Hauschka Verlag,WDK,de,W
1,104,9865,1,Ina Rometsch,moses. Verlag GmbH,YFCF,de,Y
2,104,26296,1,Agnes Spiecker,Hauschka Verlag,WDK,de,W
3,194,67241,1,Cassandra Clare,Arena Verlag GmbH,YFH,de,Y
4,194,38131,1,Cassandra Clare,Arena Verlag GmbH,YFH,de,Y


In [89]:
basket = (order_df.groupby(['sessionID', 'first topic'])['order_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))

In [90]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [94]:
!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [101]:
frequent_itemsets = apriori(basket_sets, min_support=0.003, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Y),(N),0.632565,0.004686,0.003124,0.004938,1.053909,0.00016,1.000254
1,(N),(Y),0.004686,0.632565,0.003124,0.666667,1.053909,0.00016,1.102304


In [110]:
#we will continue with basket data.
basket_df = basket_data.merge(books_data, how='left', on='itemID')
basket_df = basket_df.drop(['click', 'basket', 'order', 'click_0_1', 'order_0_1', 'Unnamed: 0','title', 'subtopics', 'description', 'interest_age', 'LangDetect', 'ISBN_13'], axis=1)

In [112]:
basket_df= basket_df[basket_df['language'] == 'de']
basket_df.dropna(axis=0, subset=['sessionID'], inplace=True)
basket_df.count() #again around 75% of all items that were in the baskets of more than one user, are written in German.

sessionID     20626
itemID        20626
basket_0_1    20626
author        17908
publisher     20626
main topic    20582
language      20626
dtype: int64

In [113]:
basket_df.dropna(subset=['main topic'])
basket_df['first topic'] = basket_df['main topic'].astype(str).str[0]
basket_df.head()

,sessionID,itemID,basket_0_1,author,publisher,main topic,language,first topic
0,7,14576,1,Julie Kagawa,Heyne Taschenbuch,YFE,de,Y
1,7,17731,1,Julie Kagawa,Heyne Taschenbuch,YFH,de,Y
2,12,29508,1,Carolin Langbein,Tessloff Verlag,YNV,de,Y
3,28,57800,1,Andrzej Sapkowski,dtv Verlagsgesellschaft,ATFN,de,A
4,28,11525,1,Andrzej Sapkowski,dtv Verlagsgesellschaft,ATFN,de,A


In [114]:
basket2 = (basket_df.groupby(['sessionID', 'first topic'])['basket_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))

In [115]:
basket_sets2 = basket2.applymap(encode_units)

In [117]:
frequent_itemsets2 = apriori(basket_sets2, min_support=0.002, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)
rules2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(J),(W),0.014938,0.082348,0.002364,0.158228,1.921442,0.001133,1.090142
1,(W),(J),0.082348,0.014938,0.002364,0.028703,1.921442,0.001133,1.014171


In [119]:
#we will continue with click data.
click_df = click_data.merge(books_data, how='left', on='itemID')
click_df = click_df.drop(['click', 'basket', 'order', 'basket_0_1', 'order_0_1', 'Unnamed: 0','title', 'subtopics', 'description', 'interest_age', 'LangDetect', 'ISBN_13'], axis=1)

In [120]:
click_df= click_df[click_df['language'] == 'de']
click_df.dropna(axis=0, subset=['sessionID'], inplace=True)
click_df.count() #again around 77% of all items that were clicked by more than one user, are written in German.

sessionID     87460
itemID        87460
click_0_1     87460
author        77233
publisher     87460
main topic    87259
language      87460
dtype: int64

In [121]:
click_df.dropna(subset=['main topic'])
click_df['first topic'] = click_df['main topic'].astype(str).str[0]
click_df.head()

,sessionID,itemID,click_0_1,author,publisher,main topic,language,first topic
0,7,14576,1,Julie Kagawa,Heyne Taschenbuch,YFE,de,Y
1,7,17731,1,Julie Kagawa,Heyne Taschenbuch,YFH,de,Y
2,12,30277,1,Lorena Lehnert,Tessloff Verlag,YNV,de,Y
3,12,29508,1,Carolin Langbein,Tessloff Verlag,YNV,de,Y
4,12,75659,1,Lorena Lehnert,Tessloff Verlag,YNWD1,de,Y


In [122]:
basket3 = (click_df.groupby(['sessionID', 'first topic'])['click_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))

In [123]:
basket_sets3 = basket3.applymap(encode_units)

In [127]:
frequent_itemsets3 = apriori(basket_sets3, min_support=0.01, use_colnames=True)
rules3 = association_rules(frequent_itemsets3, metric="lift", min_threshold=1)
rules3.head()
#up to know we can conclude that, there is not any meaningful associations between different main topics.
#so we can confidently use the Lin Similarity measure that return 0 when the first letter of two main topics is different.

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(V),(F),0.025288,0.433281,0.014333,0.566802,1.308162,0.003377,1.308220
1,(F),(V),0.433281,0.025288,0.014333,0.033081,1.308162,0.003377,1.008060
2,(Y),(J),0.647464,0.022934,0.018053,0.027883,1.215824,0.003205,1.005092
3,(J),(Y),0.022934,0.647464,0.018053,0.787202,1.215824,0.003205,1.656671
4,(Y),(W),0.647464,0.106068,0.075968,0.117331,1.106186,0.007292,1.012760


In [129]:
#now we repeat our analysis w.r.t publishers and authors!
order_df = order_data.merge(books_data, how='left', on='itemID')
order_df = order_df.drop(['click', 'basket', 'order', 'click_0_1', 'basket_0_1', 'Unnamed: 0','title', 'subtopics', 'description', 'interest_age', 'LangDetect', 'ISBN_13'], axis=1)
order_df= order_df[order_df['language'] == 'de']
order_df.dropna(axis=0, subset=['sessionID'], inplace=True)
order_df.dropna(subset=['main topic'])
order_df['first topic'] = order_df['main topic'].astype(str).str[0]
order_df.head()

,sessionID,itemID,order_0_1,author,publisher,main topic,language,first topic
0,104,74094,1,Agnes Spiecker,Hauschka Verlag,WDK,de,W
1,104,9865,1,Ina Rometsch,moses. Verlag GmbH,YFCF,de,Y
2,104,26296,1,Agnes Spiecker,Hauschka Verlag,WDK,de,W
3,194,67241,1,Cassandra Clare,Arena Verlag GmbH,YFH,de,Y
4,194,38131,1,Cassandra Clare,Arena Verlag GmbH,YFH,de,Y


In [130]:
order_df.count()

sessionID      5279
itemID         5279
order_0_1      5279
author         4838
publisher      5279
main topic     5272
language       5279
first topic    5279
dtype: int64

In [131]:
topic_type = order_df.groupby('first topic',sort = False).agg({'itemID': 'count'}).reset_index()
topic_type.sort_values(by=['itemID'], ascending=False, inplace=True, ignore_index= True)
print(topic_type)

   first topic  itemID
0            Y    2932
1            F    1866
2            W     189
3            Q      68
4            A      63
5            X      55
6            V      34
7            J      30
8            N      12
9            D      11
10           S       7
11           n       7
12           R       2
13           U       2
14           M       1


In [132]:
y_df= order_df[order_df['first topic'] == 'Y']

sessionID      2932
itemID         2932
order_0_1      2932
author         2523
publisher      2932
main topic     2932
language       2932
first topic    2932
dtype: int64

In [134]:
basket5 = (y_df.groupby(['sessionID', 'publisher'])['order_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))
basket_sets5 = basket5.applymap(encode_units)
frequent_itemsets5 = apriori(basket_sets5, min_support=0.001, use_colnames=True)
rules5 = association_rules(frequent_itemsets5, metric="lift", min_threshold=1)
rules5.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AMEET Verlag),(Oetinger Taschenbuch GmbH),0.006173,0.042593,0.001235,0.200000,4.695652,0.000972,1.196759
1,(Oetinger Taschenbuch GmbH),(AMEET Verlag),0.042593,0.006173,0.001235,0.028986,4.695652,0.000972,1.023494
2,(Coppenrath F),(Arena Verlag GmbH),0.033951,0.043827,0.001852,0.054545,1.244558,0.000364,1.011337
3,(Arena Verlag GmbH),(Coppenrath F),0.043827,0.033951,0.001852,0.042254,1.244558,0.000364,1.008669
4,(Frech Verlag GmbH),(Arena Verlag GmbH),0.016667,0.043827,0.001235,0.074074,1.690141,0.000504,1.032667


In [135]:
basket6 = (y_df.groupby(['sessionID', 'author'])['order_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))
basket_sets6 = basket6.applymap(encode_units)
frequent_itemsets6 = apriori(basket_sets6, min_support=0.001, use_colnames=True)
rules6 = association_rules(frequent_itemsets6, metric="lift", min_threshold=1)
rules6.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Doris Rübel),(Andrea Erne),0.005583,0.011863,0.001396,0.250000,21.073529,0.001329,1.317516
1,(Andrea Erne),(Doris Rübel),0.011863,0.005583,0.001396,0.117647,21.073529,0.001329,1.127006
2,(Andrea Erne),(Katja Reider),0.011863,0.004187,0.002791,0.235294,56.196078,0.002742,1.302217
3,(Katja Reider),(Andrea Erne),0.004187,0.011863,0.002791,0.666667,56.196078,0.002742,2.964410
4,(Andrea Erne),(Patricia Mennen),0.011863,0.006281,0.002791,0.235294,37.464052,0.002717,1.299479


In [136]:
f_df= order_df[order_df['first topic'] == 'F']
basket7 = (f_df.groupby(['sessionID', 'publisher'])['order_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))
basket_sets7 = basket7.applymap(encode_units)
frequent_itemsets7 = apriori(basket_sets7, min_support=0.001, use_colnames=True)
rules7 = association_rules(frequent_itemsets7, metric="lift", min_threshold=1)
rules7.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Penhaligon),(Anaconda Verlag),0.063224,0.005269,0.001054,0.016667,3.163333,0.000721,1.011591
1,(Anaconda Verlag),(Penhaligon),0.005269,0.063224,0.001054,0.200000,3.163333,0.000721,1.170969
2,(Bastei Lübbe),(Lübbe),0.005269,0.073762,0.001054,0.200000,2.711429,0.000665,1.157798
3,(Lübbe),(Bastei Lübbe),0.073762,0.005269,0.001054,0.014286,2.711429,0.000665,1.009148
4,(Bastei Lübbe),(epubli),0.005269,0.003161,0.001054,0.200000,63.266667,0.001037,1.246048


In [139]:
basket8 = (f_df.groupby(['sessionID', 'author'])['order_0_1']
          .sum().unstack().reset_index().fillna(0)
          .set_index('sessionID'))
basket_sets8 = basket8.applymap(encode_units)
frequent_itemsets8 = apriori(basket_sets8, min_support=0.001, use_colnames=True)
rules8 = association_rules(frequent_itemsets8, metric="lift", min_threshold=1)
rules8.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Adrian Tchaikovsky),(Louisa S. Reinwarth),0.001056,0.001056,0.001056,1.000,947.000000,0.001055,inf
1,(Louisa S. Reinwarth),(Adrian Tchaikovsky),0.001056,0.001056,0.001056,1.000,947.000000,0.001055,inf
2,(Amanda Bouchet),(Akram El-Bahay),0.005280,0.008448,0.001056,0.200,23.675000,0.001011,1.239440
3,(Akram El-Bahay),(Amanda Bouchet),0.008448,0.005280,0.001056,0.125,23.675000,0.001011,1.136823
4,(Akram El-Bahay),(Ben Aaronovitch),0.008448,0.035903,0.002112,0.250,6.963235,0.001809,1.285463
